# Load the dataset
Don't change this code


In [ ]:
# Load and preprocess the data. Don't change this code
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import pickle


# CIFAR10 z-normalization const https://github.com/facebookarchive/fb.resnet.torch/issues/180
cifar10_mean = (0.491, 0.482, 0.447)
cifar10_std = (0.247, 0.244, 0.262)

# Data preprocessing
transform=transforms.Compose([
                              transforms.ToTensor(), # PIL Image to Pytorch tensor
                              transforms.Normalize(cifar10_mean, cifar10_std) # https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transforms%20normalize#torchvision.transforms.Normalize
                              ])

dataset = datasets.CIFAR10("content", train=True, transform = transform ,  download=True)

# Load class names
with open("content/cifar-10-batches-py/batches.meta",'rb') as infile:
  cifar_meta = pickle.load(infile)
labels = cifar_meta['label_names']

# Split dataset into train and val
train_ds, val_ds, _= random_split(dataset, [10000, 2000 ,38000])
batch_size = 256

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size = batch_size)
val_loader = DataLoader(val_ds, batch_size = batch_size)


# Function for accuracy checking

Don't change this code

In [ ]:
def validate(model,testloader, device = "cpu"):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
  
  return correct / total  

# Implement CNN class for CIFAR10

**In constructor**

Define 2 - 3 convolutional layers 

 https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

with corresponding in/out dimensions W_out = 1 + ((W_in - F + 2*P) / S)


Also define max pooling : https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

and fully connected layers: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear


**In forward**

Write code for forward pass.
Remember that first dimension is the batch dimension

In [ ]:
import torch.nn as nn
nn.Conv2d?

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class TwoLayerCNN(nn.Module):

    def __init__(self, class_nums = 10):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16 * 3, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16 * 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16 * 3, 32 * 3, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32 * 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2))
        
        self.fc = nn.Linear(2400, class_nums)

    def forward(self, batch):
        scores = self.layer1(batch)
        scores = self.layer2(scores)
        scores = scores.reshape(scores.size(0), -1)
        scores = self.fc(scores)
        return scores


# Train the model




## First Activate tensorboard extension

Use summaryWriter to create logs: https://pytorch.org/docs/stable/tensorboard.html?highlight=summarywriter#torch.utils.tensorboard.writer.SummaryWriter

Display loss and accuracy charts.

You can cange log dir name.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## Implement training loop

- Create optimizer,
- Save loss and accuracy values into tensorboard log
- Use GPU to speedup training process.


In [ ]:
import torch.optim as optim
import torchvision
import os
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TwoLayerCNN(10)
model = model.to(device)
model.train()

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(15):
  for step, (img_batch, labels_batch) in enumerate(train_loader):
    img_batch = img_batch.to(device)
    labels_batch = labels_batch.to(device)

    output = model(img_batch)
    loss = criterion(output, labels_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()   
  
  accuracy = validate(model, val_loader, device)

  writer.add_scalar('Accuracy/train', accuracy, epoch)
  writer.add_scalar('Loss/train', loss.item(), epoch)

## Validat results on test dataset

You must get accuracy above 0.65

In [ ]:
test_dataset = datasets.CIFAR10("content",
                           train=False,
                           transform = dataset.transform, # Transforms stay the same
                           download=True)

test_loader = DataLoader(test_dataset, batch_size = batch_size)

accuracy = validate(model,test_loader,device)
print(f"Accuracy on test:{accuracy}")


Files already downloaded and verified
Accuracy on test:0.6485


# Ideas for extra work

---
1. Evaluate the impact of the number and size of filters in convolutional layers on the accuracy.

2. Evaluate the impact of the convolutional layers count on the accuracy.


3. Visualization something ...

---







